In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
import torchtext
from torchtext.data.utils import get_tokenizer

MAX_LENGTH = 20

#テキストに処理を行うFieldを定義
#fix_lengthはtokenの数
SRC = torchtext.data.Field(sequential=True, use_vocab=True, 
                            lower=True, include_lengths=True, batch_first=True, fix_length=MAX_LENGTH,
                            eos_token='<eos>')

TRG = torchtext.data.Field(sequential=True, use_vocab=True, 
                            lower=True, include_lengths=True, batch_first=True, fix_length=MAX_LENGTH,
                            init_token='<cls>', eos_token='<eos>')

#pandasでcsvを保存するときに、labelをintでキャストしておかないとエラーでるから注意
train_ds, val_ds = torchtext.data.TabularDataset.splits(
    path='/content/drive/My Drive/dataset/TIU/twitter/5_20', train='train.csv', validation='val.csv', 
    format='csv', fields=[('src', SRC), ('trg', TRG)])

In [20]:
#学習済みの分散表現をロードする
from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='/content/drive/My Drive/embedding/japanese_fasatext/model.vec')

print(japanese_fasttext_vectors.dim)
print(len(japanese_fasttext_vectors.itos))

300
351122


In [21]:
SRC.build_vocab(train_ds, vectors=japanese_fasttext_vectors)
TRG.build_vocab(train_ds, vectors=japanese_fasttext_vectors)
#SRC.build_vocab(train_ds)
#TRG.build_vocab(train_ds)
print(TRG.vocab.stoi)
print(len(TRG.vocab.stoi))

defaultdict(<function _default_unk_index at 0x7f37d38f7620>, {'<unk>': 0, '<pad>': 1, '<cls>': 2, '<eos>': 3, '。': 4, 'です': 5, '、': 6, '私': 7, 'は': 8, 'の': 9, 'て': 10, 'も': 11, 'た': 12, 'ね': 13, '!': 14, 'に': 15, 'が': 16, 'ます': 17, 'で': 18, 'よ': 19, 'し': 20, 'か': 21, 'な': 22, 'ない': 23, 'と': 24, 'ん': 25, '?': 26, 'だ': 27, '...': 28, 'から': 29, '!!': 30, '目': 31, '口': 32, 'ござい': 33, 'ふた': 34, 'ありがとう': 35, 'を': 36, 'てる': 37, 'さん': 38, 'まし': 39, '笑': 40, 'いい': 41, 'そう': 42, 'w': 43, 'けど': 44, 'ので': 45, 'たら': 46, 'たい': 47, '好き': 48, 'って': 49, 'お': 50, 'う': 51, 'ば': 52, 'ちゃん': 53, '嬉しい': 54, '大丈夫': 55, 'それ': 56, 'さ': 57, 'こと': 58, 'でも': 59, 'ください': 60, 'わ': 61, '!!!': 62, 'れ': 63, '今日': 64, 'え': 65, 'する': 66, 'なっ': 67, '見': 68, 'これ': 69, 'い': 70, 'なんて': 71, 'www': 72, '......': 73, 'とか': 74, 'あ': 75, 'ましょ': 76, '??': 77, 'だけ': 78, 'やっ': 79, '...。': 80, 'ある': 81, 'や': 82, 'なら': 83, '呼ん': 84, '呼び': 85, 'じゃ': 86, '何': 87, 'ませ': 88, '人': 89, 'もう': 90, 'ww': 91, '方': 92, 'でし': 93, '今': 94, '可愛い': 

In [22]:
from torchtext import data

batch_size = 128

train_dl = data.Iterator(train_ds, batch_size=batch_size, train=True)
val_dl = data.Iterator(val_ds, batch_size=batch_size, train=False, sort=False)
batch = next(iter(val_dl))
print(batch.src[0].shape)
print(batch.trg[0].shape)

torch.Size([128, 20])
torch.Size([128, 20])


In [23]:
class EncoderRNN(nn.Module):
  def __init__(self, emb_size, hidden_size, vocab_size, text_embedding_vectors, dropout=0):
    super(EncoderRNN, self).__init__()
    self.hidden_size = hidden_size
    if text_embedding_vectors == None:
      self.embedding = nn.Embedding(vocab_size, emb_size)
    else:
      self.embedding = nn.Embedding.from_pretrained(
          embeddings=text_embedding_vectors, freeze=True)
    self.lstm = nn.LSTM(emb_size, hidden_size, batch_first=True, bidirectional=True)
    self.thought = nn.Linear(hidden_size*2, hidden_size)


  def forward(self, input_seq, hidden=None):
    embedded = self.embedding(input_seq) #[64, 30, 600]
    outputs, (hn, cn) = self.lstm(embedded) #[64, 30, 1200], ([2, 64, 600], [2, 64, 600])
    outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] #[64, 30, 600]
    thought_vector = torch.cat((hn[0], hn[1]), -1) #[64, 1200]
    thought_vector = self.thought(thought_vector).unsqueeze(0) #[1, 64, 600]

    return outputs, thought_vector

In [24]:
#エンコーダテスト
emb_size = 300
hidden_size = 600
dropout = 0.1

batch = next(iter(val_dl))
print(batch.src[0].shape)

encoder = EncoderRNN(emb_size, hidden_size, len(SRC.vocab.stoi), SRC.vocab.vectors, dropout)
encoder_outputs, thought_vector = encoder(batch.src[0])
print(thought_vector.shape)

torch.Size([128, 20])
torch.Size([1, 128, 600])


In [25]:
class LuongAttnDecoderRNN(nn.Module):
  def __init__(self, emb_size, hidden_size, text_embedding_vectors, output_size, dropout=0.1):
    super(LuongAttnDecoderRNN, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.dropout = dropout
    if text_embedding_vectors == None:
      self.embedding = nn.Embedding(output_size, emb_size)
    else:
      self.embedding = nn.Embedding.from_pretrained(
          embeddings=text_embedding_vectors, freeze=True)
    self.embedding_dropout = nn.Dropout(dropout)
    self.lstm = nn.LSTM(emb_size, hidden_size, batch_first=True)
    self.score = nn.Linear(hidden_size, hidden_size)
    self.concat = nn.Linear(hidden_size * 2, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    
  def forward(self, input_step, decoder_hidden, encoder_outputs):
    embedded = self.embedding(input_step)
    embedded = self.embedding_dropout(embedded)
    embedded = embedded.unsqueeze(1) #[64, 1, 600]
    
    #記憶セルはencoderから引っ張ってこない
    rnn_output, hidden = self.lstm(embedded, decoder_hidden) #[64, 1, 600] ([1, 64, 600], [1, 64, 600])
    energy = self.score(encoder_outputs) # [64, 30, 600]
    attn_weights = torch.sum(rnn_output*energy, dim=2) #[64, 30]
    attn_weights = F.softmax(attn_weights, dim=1).unsqueeze(1) # [64, 1, 30]

    context = attn_weights.bmm(encoder_outputs) #[64, 1, 500]
    rnn_output = rnn_output.squeeze(1) #[64, 500]
    context = context.squeeze(1) #[64, 500]
    concat_input = torch.cat((rnn_output, context), 1) #[64, 1000]
    concat_output = torch.tanh(self.concat(concat_input))
    output = self.out(concat_output)
    output = F.softmax(output, dim=1)

    return output, hidden

In [26]:
def binaryMatrix(l, value=TRG.vocab.stoi['<pad>']):
    m = []
    for i, seq in enumerate(l):
      if seq == TRG.vocab.stoi['<pad>']:
        m.append(False)
      else:
        m.append(True)
    return m

def maskNLLLoss(inp, target):
    mask = target
    mask = binaryMatrix(mask)
    mask = torch.BoolTensor(mask)
    mask = mask.to(device)
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [27]:
decoder = LuongAttnDecoderRNN(emb_size, hidden_size, TRG.vocab.vectors, len(TRG.vocab.stoi),  dropout)
decoder_input = torch.LongTensor([TRG.vocab.stoi['<cls>'] for _ in range(batch_size)])
embedding = nn.Embedding(len(SRC.vocab.stoi), hidden_size)
print(decoder_input.shape)

cn = torch.zeros(1, batch_size, hidden_size)
#cn = torch.zeros(1, batch_size, hidden_size, device=device)
decoder_hidden = (thought_vector, cn)
target_variable = batch.trg[0]


for t in range(MAX_LENGTH):
  decoder_output, decoder_hidden = decoder(
      decoder_input, decoder_hidden, encoder_outputs
  ) #[64, 単語種類数], [2, 64, 500]
  # Teacher forcing: next input is current target
  _, topi = decoder_output.topk(1)
  decoder_input = torch.LongTensor([topi[i] for i in range(batch_size)])
  mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[:, t])
  
decoder_output, decoder_hidden = decoder(
    decoder_input, decoder_hidden, encoder_outputs
) #[64, 単語種類数], [2, 64, 500]

print('decoder_output', decoder_output.shape)
print('decoder_hidden', decoder_hidden[0].shape)

torch.Size([128])
decoder_output torch.Size([128, 21971])
decoder_hidden torch.Size([1, 128, 600])


In [28]:
def a_batch_loss(input_variable, target_variable, max_target_len, encoder, decoder, 
                 encoder_optimizer, decoder_optimizer, phase):
  total_loss = 0 #1batchのloss
  # Zero gradients
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()
  n_totals = 0
  print_losses = []
  
  #エンコーダの出力
  encoder_outputs, thought_vector = encoder(input_variable)
  #['<cls>']を生成
  decoder_input = torch.LongTensor([TRG.vocab.stoi['<cls>'] for _ in range(batch_size)]) #[64]
  decoder_input = decoder_input.to(device)
  #エンコーダの最後の隠れ状態を使用、記憶セルは0を入力
  cn = torch.zeros(1, batch_size, hidden_size, device=device)
  decoder_hidden = (thought_vector, cn)

  #teaching_forceを使う
  loss = 0 #1batchの中の1センテンスのloss
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

  if use_teacher_forcing:
    for t in range(max_target_len - 1):
      decoder_output, decoder_hidden = decoder(
          decoder_input, decoder_hidden, encoder_outputs
      ) #[64, 単語種類数], [2, 64, 500]
      decoder_input = target_variable[:, t] #[64], teaching_forceの場合、正解データを次に入力する
      #loss += criterion(decoder_output, target_variable[:, t])
      #各バッチのtのlossをだす。mask_lossはnTotalで割った平均、nTotalはバッチ数からmask(<pad>)の数を引いたもの
      mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[:, t])
      loss += mask_loss
      print_losses.append(mask_loss.item() * nTotal)
      n_totals += nTotal
    #total_loss += loss / max_target_len #1バッチ分のloss
    
  else:
    for t in range(max_target_len - 1):
      decoder_output, decoder_hidden = decoder(
          decoder_input, decoder_hidden, encoder_outputs
      ) #[64, 単語種類数], [2, 64, 500]
      _, topi = decoder_output.topk(1)
      decoder_input = torch.LongTensor([topi[i] for i in range(batch_size)])
      decoder_input = decoder_input.to(device)
      #loss += criterion(decoder_output, target_variable[:, t])
      mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[:, t])
      loss += mask_loss
      print_losses.append(mask_loss.item() * nTotal)
      n_totals += nTotal
    #total_loss += (loss / max_target_len) #1バッチ分のloss
    
  if phase == 'train':
    loss.backward()
    #total_loss.backward()
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    encoder_optimizer.step()
    decoder_optimizer.step()
  return sum(print_losses) / n_totals
  #return total_loss #1バッチ分のloss

In [29]:
import random

def train_model(dataloaders_dict, num_epochs, encoder, decoder, encoder_optimizer, decoder_optimizer):
  print("Training...")
  #エポック
  for epoch in range(num_epochs):
    for phase in ['train', 'val']:
      if phase == 'train':
        encoder.train()
        decoder.train()
      else:
        encoder.eval()
        decoder.eval()
      print_loss = 0 #1epochのloss

      for i, batch in enumerate(dataloaders_dict[phase]): 
        input_variable = batch.src[0].to(device) #(64, 30)
        target_variable = batch.trg[0][:, 1:].to(device) #(64, 30)
        max_target_len = max(batch.trg[1])
        if target_variable.shape[0] == batch_size:
          total_loss = a_batch_loss(input_variable, target_variable, max_target_len, encoder, decoder, encoder_optimizer, decoder_optimizer, phase) #1バッチ分のloss     
          print_loss += total_loss #1epochのlossをprint_lossに加えていく

      #損失をだす
      print("epoch: {}; phase: {}; Average loss: {:.4f}; PPL: {:.4f}".format(epoch+1, phase, print_loss/i, math.exp(print_loss/i) ))  

In [34]:
hidden_size = 1000
dropout = 0.1

clip = 1.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_rate = 1.0
num_epochs = 10

encoder = EncoderRNN(emb_size, hidden_size, len(SRC.vocab.stoi), SRC.vocab.vectors, dropout)
decoder = LuongAttnDecoderRNN(emb_size, hidden_size, TRG.vocab.vectors, len(TRG.vocab.stoi),  dropout)

encoder = encoder.to(device)
decoder = decoder.to(device)

In [36]:
from torch import optim

dataloaders_dict = {"train": train_dl, "val": val_dl}

encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate*decoder_learning_rate )

encoder.train()
decoder.train()

LuongAttnDecoderRNN(
  (embedding): Embedding(21933, 300)
  (embedding_dropout): Dropout(p=0.1, inplace=False)
  (lstm): LSTM(300, 1000, batch_first=True)
  (score): Linear(in_features=1000, out_features=1000, bias=True)
  (concat): Linear(in_features=2000, out_features=1000, bias=True)
  (out): Linear(in_features=1000, out_features=25195, bias=True)
)

In [37]:
train_model(dataloaders_dict, num_epochs, encoder, decoder, encoder_optimizer, decoder_optimizer)

Training...
epoch: 1; phase: train; Average loss: 6.4630; PPL: 641.0084
epoch: 1; phase: val; Average loss: 5.7918; PPL: 327.5873
epoch: 2; phase: train; Average loss: 5.5509; PPL: 257.4622
epoch: 2; phase: val; Average loss: 5.4454; PPL: 231.6963
epoch: 3; phase: train; Average loss: 5.2491; PPL: 190.3891
epoch: 3; phase: val; Average loss: 5.2322; PPL: 187.2040
epoch: 4; phase: train; Average loss: 5.0617; PPL: 157.8577
epoch: 4; phase: val; Average loss: 5.1165; PPL: 166.7466
epoch: 5; phase: train; Average loss: 4.9391; PPL: 139.6500
epoch: 5; phase: val; Average loss: 5.0470; PPL: 155.5522
epoch: 6; phase: train; Average loss: 4.8460; PPL: 127.2249
epoch: 6; phase: val; Average loss: 4.9840; PPL: 146.0600
epoch: 7; phase: train; Average loss: 4.7691; PPL: 117.8158
epoch: 7; phase: val; Average loss: 4.9503; PPL: 141.2128
epoch: 8; phase: train; Average loss: 4.7004; PPL: 109.9961
epoch: 8; phase: val; Average loss: 4.9215; PPL: 137.2060
epoch: 9; phase: train; Average loss: 4.6370